In [3]:
import os

import gradio as gr
import tensorflow as tf
import cv2
import numpy as np

from PIL import Image
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import gradio as gr


In [10]:
unet_model_path="Modelfile/unet_custom.hdf5"
os.path.exists(unet_model_path)
model = tf.keras.models.load_model("unet_model_path")

OSError: No file or directory found at unet_model_path

In [12]:
from utils_unet_a.py import *

ModuleNotFoundError: No module named 'imgaug'

In [11]:
import gradio as gr
import tensorflow as tf
import cv2
import numpy as np

# Load the UNet model
model = tf.keras.models.load_model("unet_custom.hdf5")

# Define the image segmentation function
def segment_head_circumference(image):
    # Preprocess the input image
    processed_image = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    processed_image = processed_image / 255.0
    processed_image = np.expand_dims(processed_image, axis=0)
    processed_image = np.expand_dims(processed_image, axis=-1)

    # Perform the segmentation
    prediction = model.predict(processed_image)
    prediction = np.squeeze(prediction)
    binary_image = np.where(prediction > 0.5, 255, 0).astype(np.uint8)

    return binary_image

# Define the input and output interfaces for Gradio
inputs = gr.inputs.Image(label="Ultrasound Image (PNG format)")
output = gr.outputs.Image(label="Segmented Head Circumference")

# Create the Gradio interface
gr_interface = gr.Interface(
    fn=segment_head_circumference,
    inputs=inputs,
    outputs=output,
    title="Ultrasound Image Segmentation",
    description="Segment the head circumference in ultrasound images.",
)

# Run the Gradio app on the Hugging Face Spaces platform
gr_interface.launch()


ValueError: Unknown loss function: bce_dice_loss. Please ensure this object is passed to the `custom_objects` argument. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [2]:
g_path_to = 'G:/.shortcut-targets-by-id/10wVB-YsfTmmpZyUo8jBH1cDOkVWVx5N1/T1 Project/Data/'
os.path.exists(g_path_to)

True

In [3]:
path_to = 'Data/Training-Splitted'
os.path.exists(path_to)


True

In [4]:
pred_img2_path_to = 'Data/predicted_images2'
os.path.exists(pred_img2_path_to)

False

In [5]:
files = os.listdir(path_to)

In [6]:
model_path_to = 'Modelfile/'
os.path.exists(model_path_to)

True

In [7]:
rf_model_joblib_name = 'pixelCt_pixelSz_toHeadCir_rfReg.joblib'
rf_model_joblib_file = model_path_to+rf_model_joblib_name  
os.path.exists(rf_model_joblib_file)

True

In [8]:
rf_model_joblib_file

'Modelfile/pixelCt_pixelSz_toHeadCir_rfReg.joblib'

In [9]:
#import joblib

# Load the trained Random Forest model
rf_model = joblib.load(rf_model_joblib_file)

# to use rf_model.predict(input_data)   #input_data has column of feature_names = ['pixel_count','pixel size(mm)']
feature_names = ['pixel_count','pixel size(mm)']

# X = input_dffeature_names]
# y_pred  = rf_model.predict(X)

In [10]:
##### function get_pixe_count get_pixel_count(img_data_path, filename) ###  duplicate code from above.
def get_pixel_count(img_data_path, filename):
    file_path = os.path.join(img_data_path, filename)
    if os.path.exists(file_path):
        label = Image.open(file_path)
        label = np.array(label)
        label = tf.convert_to_tensor(label)
        label = tf.cast(label, tf.float32) / 255.0
        label = tf.cast(label, tf.int32)
        pixel_count = np.sum(label)
    else:
        print("No such file: " + filename)
        pixel_count = np.nan

    return pixel_count


In [11]:
### get_pixel_count_gradio demo ###
import numpy as np
import gradio as gr

def get_pixel_count_gradio(input_img):
    label = np.array(input_img)
    label = tf.convert_to_tensor(label)
    label = tf.cast(label, tf.float32) / 255.0
    label = tf.cast(label, tf.int32)
    pixel_count = np.sum(label)
    print(pixel_count)
    return pixel_count

demo = gr.Interface(
    fn=get_pixel_count_gradio,
    inputs=gr.inputs.Image(shape=(800, 540)),
    outputs="number"
)

#demo.launch(share=True)
demo.launch()



C:\Users\annch\.conda\envs\unet-py39\lib\site-packages\gradio\inputs.py:257: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\annch\.conda\envs\unet-py39\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [12]:
import numpy as np
import gradio as gr
import joblib

# Load the RF model
#rf_model = joblib.load("rf_model_joblib_file")

### get_pixel_count_gradio demo ###
def get_pixel_count_gradio(input_img):
    label = np.array(input_img)
    pixel_count = np.sum(label)
    print(pixel_count)
    return pixel_count

demo = gr.Interface(
    fn=get_pixel_count_gradio,
    inputs=gr.inputs.Image(shape=(800, 540)),
    outputs="number"
)

demo.launch()

def predict_head_circumference(input_img, pixel_size):
    # Use the pixel size and other features to make a prediction
    feature_names = ['pixel_count', 'pixel size(mm)']  # Add other relevant features here
    pixel_count = get_pixel_count_gradio(input_img)
    
    prediction = rf_model.predict([[pixel_count, pixel_size]])[0]
    print(prediction)
    
    return prediction

inputs = [
    gr.inputs.Image(label="Image"),
    gr.inputs.Number(label="Pixel Size (mm)")
]

output = gr.outputs.Textbox(label="Head Circumference (mm)")

gr_interface = gr.Interface(
    fn=predict_head_circumference,
    inputs=inputs,
    outputs=output,
    title="Head Circumference Predictor",
    description="Predicts the head circumference based on an image and pixel size.",
)

gr_interface.launch()


C:\Users\annch\.conda\envs\unet-py39\lib\site-packages\gradio\inputs.py:257: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\annch\.conda\envs\unet-py39\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


C:\Users\annch\.conda\envs\unet-py39\lib\site-packages\gradio\inputs.py:257: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\annch\.conda\envs\unet-py39\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\annch\.conda\envs\unet-py39\lib\site-packages\gradio\inputs.py:59: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\annch\.conda\envs\unet-py39\lib\site-packages\gradio\outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


In [13]:





















#import gradio as gr
#import pandas as pd
#import joblib
#import cv2

# Load the RF model
rf_model = joblib.load(rf_model_joblib_file)

# Load the CSV file with pixel size data
#csv_file_path = 'pixel_size_data.csv'
#pixel_size_data = pd.read_csv(csv_file_path)

def predict_head_circumference(input_img, pixel_size):
    # Process the image (e.g., resize, normalize, etc.)
    #processed_image = preprocess_image(image)  #image already in binary form
    
    # Use the pixel size and other features to make a prediction
    feature_names = ['pixel_count','pixel size(mm)']  # Add other relevant features here
    pixel_count = get_pixel_count_gradio(input_img)
    pixel_size = pixel_size
    
    prediction = rf_model.predict([pixel_count,pixel_size])[0]
    print(prediction)
    
    return prediction

#def preprocess_image(image):
#    # Preprocess the image (e.g., resize, normalize, etc.)
#    # Modify this function based on your specific image preprocessing requirements
#    processed_image = cv2.resize(image, (224, 224))
#    processed_image = processed_image / 255.0  # Normalize pixel values
    
#    return processed_image

# Define the input and output interfaces for Gradio
inputs = [
    gr.inputs.Image(label="Image"),
    gr.inputs.Number(label="Pixel Size (mm)")
]

output = gr.outputs.Textbox(label="Head Circumference (mm)")

# Create the Gradio interface
gr_interface = gr.Interface(
    fn=predict_head_circumference,
    inputs=inputs,
    outputs=output,
    title="Head Circumference Predictor",
    description="Predicts the head circumference based on an image and pixel size.",
)

# Run the Gradio app
gr_interface.launch()


C:\Users\annch\.conda\envs\unet-py39\lib\site-packages\gradio\inputs.py:257: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\annch\.conda\envs\unet-py39\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\annch\.conda\envs\unet-py39\lib\site-packages\gradio\inputs.py:59: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\annch\.conda\envs\unet-py39\lib\site-packages\gradio\outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


In [14]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(
    fn=greet,
    inputs=gr.Textbox(lines=2, placeholder="Name Here..."),
    #outputs="text",
    outputs=gr.outputs.Textbox(label="Prediction")
)
demo.launch()




C:\Users\annch\.conda\envs\unet-py39\lib\site-packages\gradio\outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


In [15]:
import numpy as np
import gradio as gr

def sepia(input_img):
    sepia_filter = np.array([
        [0.393, 0.769, 0.189], 
        [0.349, 0.686, 0.168], 
        [0.272, 0.534, 0.131]
    ])
    sepia_img = input_img.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

demo = gr.Interface(sepia, gr.Image(shape=(200, 200)), "image")
demo.launch()


Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [16]:







# Get the input shape
input_shape = image.size

print("Input shape:", input_shape)


NameError: name 'image' is not defined

3603150
137.53759999999988


C:\Users\annch\.conda\envs\unet-py39\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
